# Model creation and initial testing

Here is where we shall create three different models, evaluate them and create an initial assessment. For a fuller evaluation this will have to be replicated many times over to get summary statistics but this will create the initial framework for that later work and provide an esarly indicator.

To remind, the three models we shall create are:
 - Natural model - this will have no PCA applied and all features will be given to the model to learn from
 - PCA model - this will have ALL features, regardless of any correlations, reduced together with PCA down to a lower dimensional space and then used to create a model
 - Hybrid model - this shall implement the method that we are testing to create the model.

## Load packages & data

In [1]:
#Import the various libraries and functions that we will require
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

import networkx as nx

In [2]:
#defin custome colouring function. Will be used later, in hybrid model creation. 
def color_thresh(val,thres=0.5):
    if abs(val)>=thres:
        if val<0:
            return 'background-color: red'
        else:
            return 'background-color: green'
    else:
        return ''

In [3]:
#Import the data
data_df = pd.read_csv("../Synth-Data-Creation/synth1.csv")
data_df.drop(columns=['Unnamed: 0'],inplace=True)

In [4]:
#import any of the parameters that we require
with open("../Synth-Data-Creation/params.pkl", "rb") as input_file:
    params = pickle.load(input_file)

In [5]:
#split loaded data into data and target variables
X = data_df.iloc[:,:-1]
y = data_df.iloc[:,-1]

In [6]:
#split data into training & test sets according to the split defined in the parameters 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=params['split'], random_state=1234)

<br></br>

## Natural model

First we shall create the Natural model, that is the model with out any PCA being applied and using all of the features to pre-process the models. This shall form the base line to which we can compaire the other models.

In [7]:
#Init and fit the model to the training data
nat_LR = LinearRegression()
nat_LR.fit(X_train,y_train)

LinearRegression()

Out of curiosity we print the coefficents and the intercept that the model believes it has found just to see if its close/near to those that we used to contruct the target variable

In [8]:
nat_LR.coef_ #model coefficents

array([ 0.68446515, 10.05304922,  5.34439598,  1.98726492,  4.25922843,
        0.0159577 ,  7.34271791,  5.78745202,  8.63615881,  5.95524685])

In [9]:
params['coeffs'][:-1] #creation coefficents

array([0.8548219 , 9.45614306, 4.93156789, 2.74044746, 4.57208473,
       0.24207174, 7.16864783, 5.66669495, 8.47362646, 6.49470771])

In [10]:
nat_LR.intercept_ #model intercept

4.051501515613731

In [11]:
params['coeffs'][-1] #creation intercept

4.226843490666489

On the whole we see that the two sets are relativly close, which is re-assuring.

Now we proceed to evaluate the model using the metrics we have loaded in across the three subsets of data we have (Train, Test & all) to give a full picture. The 4 metrics we shall be evaluating according to are:
 - $R^2$
 - Mean Absolute Error (MAE)
 - Mean Squared Error (MSE)
 - Mean Absolute Percentage Error (MAPE)

In [12]:
d_set = ['Train','Test','all']
res = []

#For each of our sets
for s in d_set: 
    #set out evaluation variables
    if s=='Train':
        x_set = X_train
        y_set = y_train
    elif s=='Test':
        x_set = X_test
        y_set = y_test
    elif s=='all':
        x_set = X
        y_set = y
    #evaluate our set and append the results to our results list
    res.append([
        s,
        nat_LR.score(x_set,y_set),
        mean_absolute_error(y_set,nat_LR.predict(x_set)),
        mean_squared_error(y_set,nat_LR.predict(x_set)),
        mean_absolute_percentage_error(y_set,nat_LR.predict(x_set))
    ])

In [13]:
#Transform into dataframe for ease of viewing
nat_res_df = pd.DataFrame(res,columns=['subset','R^2','MAE','MSE','MAPE']).set_index('subset')
nat_res_df

,R^2,MAE,MSE,MAPE
subset,,,,
Train,0.938015,4.101535,25.833789,0.725365
Test,0.944579,3.743841,22.559766,0.560040
all,0.939812,4.012111,25.015283,0.684034


This concludes the process for the natrual model. The other models that we shall create will follow the same proceedure 

<br></br>

## PCA model

In [14]:
#Init PCA transformer and fit to TRAINING data
pca = PCA(n_components=5)
pca.fit(X_train)

PCA(n_components=5)

In [15]:
#Transform training data
X_train_trans = pca.transform(X_train)

In [16]:
#Init model and fit to transformed training data
pca_LR = LinearRegression()
pca_LR.fit(X_train_trans,y_train)

LinearRegression()

In the Natural model we printed the coefficents to see how closely they matched the coefficents that we used to create the target data but here there is no point as there are less features in the model due to the PCA transformation.

We now proceed to evaluating this model

In [17]:
d_set = ['Train','Test','all']
res = []

#For each of our sets
for s in d_set:
    #set out evaluation variables
    if s=='Train':
        x_set = X_train_trans
        y_set = y_train
    elif s=='Test':
        x_set = pca.transform(X_test)
        y_set = y_test
    elif s=='all':
        x_set = pca.transform(X)
        y_set = y
    #evaluate our set and append the results to our results list
    res.append([
        s,
        pca_LR.score(x_set,y_set),
        mean_absolute_error(y_set,pca_LR.predict(x_set)),
        mean_squared_error(y_set,pca_LR.predict(x_set)),
        mean_absolute_percentage_error(y_set,pca_LR.predict(x_set))
    ])

In [18]:
#Transform into dataframe for ease of viewing
pca_res_df = pd.DataFrame(res,columns=['subset','R^2','MAE','MSE','MAPE']).set_index('subset')
pca_res_df

,R^2,MAE,MSE,MAPE
subset,,,,
Train,0.556617,10.580950,184.790792,2.062910
Test,0.548052,11.102770,183.971046,1.320428
all,0.555877,10.711405,184.585855,1.877290


We shall save our comments on the results untill all the models have been created and evaluated

<br></br>

## Hybrid-Model

Now we arrive at the heart of what we have set out to do. Creating and evaluating our hybrid model. for the moent we shall just create it in squential order but later we shall create a custom class/function for ease of implementation/developement.

First let us observe the data and the correlations it contains. we shall apply the colour function `color_thresh` to highlight the entries whose absolute value is above $0.75$. Red if it is negative, green if it is positive. The value $0.75$ is what will be used as our threshold, a parameter that can be varied

In [19]:
corrMat = X_train.corr()
display(corrMat.style.applymap(color_thresh, thres=0.75))

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10
f1,1.000000,0.858975,0.014112,-0.043008,0.064905,0.008296,-0.141791,-0.020367,-0.018657,0.078954
f2,0.858975,1.000000,0.022452,-0.022236,0.043580,0.026203,-0.090980,-0.006930,-0.015338,0.047891
f3,0.014112,0.022452,1.000000,0.816866,-0.390944,-0.043149,-0.030340,-0.005084,0.025913,0.044718
f4,-0.043008,-0.022236,0.816866,1.000000,-0.798717,-0.045279,0.016290,-0.005994,0.007046,0.021007
f5,0.064905,0.043580,-0.390944,-0.798717,1.000000,0.004636,-0.057792,0.016758,0.027323,-0.014113
f6,0.008296,0.026203,-0.043149,-0.045279,0.004636,1.000000,0.004129,0.072476,-0.038395,0.109527
f7,-0.141791,-0.090980,-0.030340,0.016290,-0.057792,0.004129,1.000000,0.003402,-0.015471,0.073689
f8,-0.020367,-0.006930,-0.005084,-0.005994,0.016758,0.072476,0.003402,1.000000,-0.048227,-0.003404
f9,-0.018657,-0.015338,0.025913,0.007046,0.027323,-0.038395,-0.015471,-0.048227,1.000000,0.007014
f10,0.078954,0.047891,0.044718,0.021007,-0.014113,0.109527,0.073689,-0.003404,0.007014,1.000000


This correlation matrix we are now going interpret as a adjacency matrix of a graph. Each of the nodes in the graph repesents the features, and the correlations between the features are the weights of the edges connecting the nodes. First any edge whose absolute value is below our threshold is changed to be equal to 0. In terms of graphs, this is cutting edges potentially breaking the initial graph into disconnected subgraphs.

In [20]:
corrMat[abs(corrMat)<=0.75]=0

Next we actually convert it into a graph object as now we wish to get all of the distinct components of the graph

In [21]:
G = nx.from_numpy_matrix(corrMat.values)

In [22]:
feat_to_reduce = [list(g) for g in nx.connected_components(G) if len(g)>1]
feat_to_keep = [list(g)[0] for g in nx.connected_components(G) if len(g)==1]

To illustrate we next print the isolated nodes. These are the nodes/features that have been completly cut from the rest of the graph and therefore will not be reduced and given to the model as-is

In [23]:
feat_to_keep

[5, 6, 7, 8, 9]

Next we print the list of connected compents - the groups of features that are to be reduced.

In [24]:
feat_to_reduce

[[0, 1], [2, 3, 4]]

See here we have two groups. Both of which will have a seperate PCA transformer applied to the respective features, reducing it to one dimension.

In [25]:
list_of_trained_pca = []
X_train_trans_hybrid = pd.DataFrame()

#for each grouping
for i in range(0,len(feat_to_reduce)):
    current_pca = PCA(n_components=1)                  #init the pca transformer
    current_pca.fit(X_train.iloc[:,feat_to_reduce[i]]) #fit the transformer on the current grouping
    list_of_trained_pca.append(current_pca)            #append to the list as we will need it later on
    
    #transform training data and put it into a dataframe
    X_train_trans_hybrid = pd.concat([
        X_train_trans_hybrid,pd.DataFrame(current_pca.transform(X_train.iloc[:,feat_to_reduce[i]]),columns=['pca_'+str(i)])],
        axis=1)

#add on the un-transformed features
X_train_trans_hybrid = pd.concat([X_train_trans_hybrid,X_train.iloc[:,feat_to_keep].reset_index(drop=True)],axis=1)

In [26]:
#init the model and fit to the transformed data
hybrid_LR = LinearRegression()
hybrid_LR.fit(X_train_trans_hybrid,y_train)

LinearRegression()

In [27]:
#create the transformed test data set.
X_test_trans_hybrid = pd.concat(
    [pd.DataFrame(list_of_trained_pca[i].transform(X_test.iloc[:,feat_to_reduce[i]]),columns=['pca_'+str(i)]) for i in range(0,len(feat_to_reduce))]+[X_test.iloc[:,feat_to_keep].reset_index(drop=True)]
    ,axis=1)

In [28]:
d_set = ['Train','Test','all']
res = []

#For each of our sets
for s in d_set:
    #set out evaluation variables
    if s=='Train':
        x_set = X_train_trans_hybrid
        y_set = y_train
    elif s=='Test':
        x_set = X_test_trans_hybrid
        y_set = y_test
    elif s=='all':
        x_set = pd.concat([X_train_trans_hybrid,X_test_trans_hybrid])
        y_set = pd.concat([y_train.reset_index(drop=True),y_test.reset_index(drop=True)])
    #evaluate our set and append the results to our results list
    res.append([
        s,
        hybrid_LR.score(x_set,y_set),
        mean_absolute_error(y_set,hybrid_LR.predict(x_set)),
        mean_squared_error(y_set,hybrid_LR.predict(x_set)),
        mean_absolute_percentage_error(y_set,hybrid_LR.predict(x_set))
    ])

In [29]:
hybrid_res_df = pd.DataFrame(res,columns=['subset','R^2','MAE','MSE','MAPE']).set_index('subset')
hybrid_res_df

,R^2,MAE,MSE,MAPE
subset,,,,
Train,0.813873,6.941348,77.573193,1.358226
Test,0.827783,6.753071,70.102845,0.780706
all,0.817848,6.894279,75.705606,1.213846


<br></br>

## Comparison

We have now evaluated all three of the models we set out to create on our inital synthetic data set that we created. Below we now display the results of all three models. 

In [30]:
nat_res_df

,R^2,MAE,MSE,MAPE
subset,,,,
Train,0.938015,4.101535,25.833789,0.725365
Test,0.944579,3.743841,22.559766,0.560040
all,0.939812,4.012111,25.015283,0.684034


In [31]:
hybrid_res_df

,R^2,MAE,MSE,MAPE
subset,,,,
Train,0.813873,6.941348,77.573193,1.358226
Test,0.827783,6.753071,70.102845,0.780706
all,0.817848,6.894279,75.705606,1.213846


In [32]:
pca_res_df

,R^2,MAE,MSE,MAPE
subset,,,,
Train,0.556617,10.580950,184.790792,2.062910
Test,0.548052,11.102770,183.971046,1.320428
all,0.555877,10.711405,184.585855,1.877290


As always context is very important. The results shown above are based on one synthetically created data set. The results shown above could have been obtained by chance or the synthetic dataset could be a inaccurate representation. These results are therfore preliminary and serve as a rough indication as to if the there maybe something worth further persuit.

In terms of raw results it is very clear with all of the selected metrics that the rankings of the models are:
 1. Natural
 2. Hybrid
 3. PCA

In every single entries of the respective tables the higher ranking model is better than the lower ranking method. If this data and these results were to present themself in a real world problem you would unequivalently choose the natural model. 

This makes sense as the natural model is working off all 10 features from the original feature space and has more "space" to create a more precise solution. The interesting part comes when you look at the difference between the natural model and the other two models, and them to each other. 

With the PCA model there is almost a $50%$ decrease in results across all the metrics making it the worst performing model. This does make sense when we consider it was built on 5 features which were constructed by PCA. The logical question would be if we saw similar results as we varied the number of PCA components. 

As for the hybrid-model we see that is performs about $10%$ worst than the natural model but it is significantly better than the PCA model by comparision. The next question is is this a one off or is this a consistent result. this test would need to be replicated multiple times over the the results statistically analised.

## PCA model v2

Just out of curiosity how does the pca model compare if we reduced it down to 7 features instead of 5. The same as what the hybrid model reduces to

In [33]:
#Init PCA transformer and fit to TRAINING data
pca2 = PCA(n_components=7)
pca2.fit(X_train)

PCA(n_components=7)

In [34]:
#Transform training data
X_train_trans = pca2.transform(X_train)

In [35]:
#Init model and fit to transformed training data
pca2_LR = LinearRegression()
pca2_LR.fit(X_train_trans,y_train)

LinearRegression()

In [36]:
d_set = ['Train','Test','all']
res = []

#For each of our sets
for s in d_set:
    #set out evaluation variables
    if s=='Train':
        x_set = X_train_trans
        y_set = y_train
    elif s=='Test':
        x_set = pca2.transform(X_test)
        y_set = y_test
    elif s=='all':
        x_set = pca2.transform(X)
        y_set = y
    #evaluate our set and append the results to our results list
    res.append([
        s,
        pca2_LR.score(x_set,y_set),
        mean_absolute_error(y_set,pca2_LR.predict(x_set)),
        mean_squared_error(y_set,pca2_LR.predict(x_set)),
        mean_absolute_percentage_error(y_set,pca2_LR.predict(x_set))
    ])

In [37]:
#Transform into dataframe for ease of viewing
pca2_res_df = pd.DataFrame(res,columns=['subset','R^2','MAE','MSE','MAPE']).set_index('subset')
pca2_res_df

,R^2,MAE,MSE,MAPE
subset,,,,
Train,0.914959,4.747275,35.442845,0.964181
Test,0.911440,4.774319,36.049371,0.668264
all,0.914358,4.754036,35.594476,0.890202


Well... i guess that answers that then...